In [3]:
import numpy as np

Constants declared for different penalties and rewards.  
NOTE: Rewards greater than or equal to 10000 can cause the q-table to overflow. So, the gold reward has been taken as 1000 rather than 10000 to avoid this.

In [4]:
SIZE = 10 # size of matrix
MOVE_PENALTY = 1 # move penalty of every move made
METAL_DETECTOR_PENALTY = 100 # penalty of getting in range of a metal detector
GOLD_REWARD = 1000 # reward for reaching the goal

The coordinates of the the robot, gold and the metal detectors are mapped from the given question.  
The coordinate systems origin is taken as the upper-left most space.

In [5]:
robot_coordinate = ((0, 9))
metal_detector_coordinates = ((1, 4), (2, 9), (3, 2), (3, 5), (7, 3), (7, 8), (8, 1))
gold_coordinate = ((9, 9))

A function is desinged to calculate the surroundings of the metal detectors.

In [6]:
def calculate_surrounding_coordinates(input_coordinates):
    surrounding_coordinates = []
    for coordinate in input_coordinates:
        for x_offset in range(-1, 2):
            for y_offset in range(-1, 2):
                surrounding_coordinates.append((coordinate[0]+x_offset, coordinate[1]+y_offset))
    return surrounding_coordinates

A robot class is being defined in a similar way to the openAI's gym environment.  
Whenever the robot wants to make a move, the step function is initiated with the action that the robot wants to take as the parameter.  
The actions are as follows  
step(0) - go right  
step(1) - go left  
step(2) - go down  
step(3) - go up  
If the robot tries to move out of the map, it stays in the same positions but gets a negative reward of -1 for taking a step.  
When the step function is initiated, it returns 3 values:  
1. Updated robots position  
2. Positive/Negative reward based on the location it is in  
3. Whether the robot has reached the goal or not  

In [7]:
class Robot:
    def __init__(self):
        self.x, self.y = robot_coordinate

    def step(self, choice):
        if choice == 0:
            return self.move(1, 0)
        elif choice == 1:
            return self.move(-1, 0)
        elif choice == 2:
            return self.move(0, 1)
        elif choice == 3:
            return self.move(0, -1)

    def move(self, x=0, y=0):
        self.x += x
        self.y += y

        if self.x < 0:
            self.x = 0
        elif self.x > SIZE - 1:
            self.x = SIZE - 1
        if self.y < 0:
            self.y = 0
        elif self.y > SIZE - 1:
            self.y = SIZE - 1

        return (self.x, self.y), self.calculate_reward(), self.done()

    def calculate_reward(self):
        if (self.x, self.y) == gold_coordinate:
            return GOLD_REWARD - MOVE_PENALTY
        elif (self.x, self.y) in calculate_surrounding_coordinates(metal_detector_coordinates):
            return - METAL_DETECTOR_PENALTY - MOVE_PENALTY
        else:
            return - MOVE_PENALTY

    def done(self):
        if (self.x, self.y) == gold_coordinate:
            return True
        else:
            return False
    
    def reset(self):
        self.x, self.y = robot_coordinate
        return self.x, self.y

In [8]:
env = Robot() # constructor for creating the robot

A function which return the q-table after the agent has gone through a given number of episodes with the given parameters.
The following parameters are passed to the function in addition to the robot object and the number of episodes the robot should train for:
1. Discounting factor - y - this decreases the impact of future rewards on the immediate decision making - the greater the discount factor, the more the robot tries to get immediate rewards.
2. Epsilon - eps - Allows the robot the explore more
3. Learning rate - lr - Q-table learning rate
4. Decay factor - decay_factor - Rate of exploration is reduced by this factor
  
parameters = (y = 0.95, eps = 0.5, lr = 0.8, decay_factor = 0.999)

In [9]:
def eps_greedy_q_learning_with_table(env, num_episodes=500, parameters = (0.95, 0.5, 0.8, 0.999)):
    q_table = np.zeros((4, 10, 10), dtype= np.int64)
    y, eps, lr, decay_factor = parameters
    for i in range(num_episodes):
        s = env.reset()
        eps *= decay_factor
        done = False
        while not done:
            # select the action with highest cummulative reward
            if np.random.random() < eps or np.sum(q_table[:, s[1], s[0]]) == 0:
                a = np.random.randint(0, 4)
            else:
                a = np.argmax(q_table[:, s[1], s[0]])
            # pdb.set_trace()
            new_s, r, done = env.step(a)
            q_table[a][s[1]][s[0]] += r + lr * (y * np.max(q_table[:, new_s[1], new_s[0]]) - q_table[a][s[1]][s[0]])
            s = new_s
    return q_table

Function to display the path that the robot takes using the q-table. Reterns the path as well as whether the robot was able to reach the goal or not.  
If the robot tries to go in an infinite loop, then that path is not accepted.

In [10]:
def display_path_from_q_table(q_table):
    coordinate = list(robot_coordinate)
    path = np.zeros((10, 10), dtype= np.int8)
    while coordinate != list(gold_coordinate):
        x = 0
        y = 0
        
        if path[coordinate[1]][coordinate[0]] != 99:
            path[coordinate[1]][coordinate[0]] = 99
        else:
            # to prevent an infinite loop, the code breaks when the program tries to go into an infinite loop
            return path, False
            
        a = np.argmax(q_table[:, coordinate[1], coordinate[0]])

        if a == 0:
            y = 1
        elif a == 1:
            y = -1
        elif a == 2:
            x = 1
        elif a == 3:
            x = -1
    
        coordinate[1] += x
        coordinate[0] += y
        
        if coordinate[1] < 0:
            coordinate[1] = 0
        elif coordinate[1] > SIZE - 1:
            coordinate[1] = SIZE - 1
        if coordinate[0]< 0:
            coordinate[0] = 0
        elif coordinate[0] > SIZE - 1:
            coordinate[0] = SIZE - 1
    
    for a in metal_detector_coordinates:
        path[a[1]][a[0]] = 22
    
    path[gold_coordinate[1]][gold_coordinate[0]] = 77
    return path, True

Parameter sets for finding which sets of parameters are the best.

In [11]:
#parameter_set = (y = 0.95, eps = 0.5, lr = 0.8, decay_factor = 0.999)
parameter_set = (0.95, 0.5, 0.8, 0.999)
parameter_set_1 = (0.85, 0.5, 0.8, 0.999)
parameter_set_2 = (0.95, 0.1, 0.8, 0.999)
parameter_set_3 = (0.95, 0.9, 0.8, 0.999)
parameter_set_4 = (0.95, 0.5, 0.3, 0.999)
parameter_set_5 = (0.95, 0.5, 0.8, 0.699)
parameter_sets = (parameter_set_1, parameter_set_2, parameter_set_3, parameter_set_4, parameter_set_5)

The below function takes the above parameters and returns the following:
1. The fastest solution, number of iterations it took to find the fastest solution and the parameter set which gave the fastest solution. The fastest path refers to the path which the algorithm was able to find with the least amount of iterations.
2. The best solution, number of iterations it took to find the best solution and the parameter set which gave the best solution. The best path refers to the path which the algorithm was able to find which will give the maximum possible rewards.

In [12]:
def find_best_set_of_parameters(parameter_sets):
    solution_found = False
    best_solution_found = False
    SOLUTION_FOUND = False
    i = 0
    q_table = eps_greedy_q_learning_with_table(env, 250, parameter_set)
    ACTUAL_BEST_SOLUTION, _ = display_path_from_q_table(q_table)
    while not solution_found or not best_solution_found:
        i += 1
        for p in parameter_sets:
            q_table = eps_greedy_q_learning_with_table(env, i, p)
            path, solution_found = display_path_from_q_table(q_table)
            if solution_found and not SOLUTION_FOUND:
                solution, fastestIteration, fastestSet = (path, i, p)
                SOLUTION_FOUND = True
            if np.array_equal(path, ACTUAL_BEST_SOLUTION):
                best_solution_found = True
                best_solution, bestIteration, bestSet = (path, i, p)
                break
    
    return (solution, fastestIteration, fastestSet), (best_solution, bestIteration, bestSet)

Running the below line will give the fastest and the best possible paths.  
The tiles in which the robot goes through is represented by 99.  
The tiles in which the metal detectors are present are represented by 22.  
The tile in which the goal is present is represented by 77.  
  
NOTE: If the path goes through a metal detector, the tile remains as 22 so that it is visible that there is a metal detector there.  

In [13]:
find_best_set_of_parameters(parameter_sets)

((array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0,  0, 22,  0],
         [ 0,  0,  0, 22,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0, 22,  0,  0],
         [ 0, 22,  0,  0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0, 22,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [99, 99, 99, 99, 99,  0,  0,  0,  0,  0],
         [99,  0,  0,  0, 99, 99, 99, 22, 99, 99],
         [99,  0, 22,  0,  0,  0,  0,  0,  0, 77]], dtype=int8),
  14,
  (0.95, 0.9, 0.8, 0.999)),
 (array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0,  0, 22,  0],
         [ 0,  0,  0, 22,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0, 22,  0,  0],
         [ 0, 22,  0,  0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0, 22,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0, 99, 99, 99, 99, 99],
         [99, 99, 99, 99, 99, 99,  0,  0,  0, 99],
         [99,  0,  0,  0,  0,  0, 